# Authors
Ikram Kohil, 2019115 \
Johnatan Gao, 2013298

# Introduction

<div style="text-align: justify">
For this first part of the lab, we were tasked with finding a pattern (calls/queries to an SQL database) in the provided Wordpress source code. To do so, we implemented a visitor that would visit all the ast generated from the source code, parse them, and prints in an output file what specific function was called at which line in which file. The following sections detail the steps undertaken to achieve the desired result.
</div>

# 1. Pattern Detection

## 1.1 Retrieve the names of all ast files present in given directory

In [ ]:
from code_analysis import AST, ASTReader
from pathlib import Path
import os

# Global variable - Reader that will read a given ast.json file and generate an ast out of it
reader = ASTReader()

def create_output_file(filename):
   # Check if output file already exists, if so, delete and create new file
   if os.path.exists(filename):
      os.remove(filename)

   # Open in 'append' mode to avoid overwriting the whole file after each modification
   return open(filename, "a")

def close_output_file(file):
   file.close()

# In specified directory, find all files ending with '.ast.json' in order to retrieve the list of all asts
# Store file names satisfying the condition in an array in order to iterate over it when calling the visitor
## Adapted from: https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
def get_ast_json_files(directory):
   directory = Path(directory)
   return [str(file) for file in directory.rglob('*.ast.json')]


## 1.2 Create visitor that will visit a given ast in search of calls to methods provided as parameter in its constructor

<div style="text-align: justify">

This class was originally made specifically to parse an ast in search of SQL queries, but was made more generic in order to be reused in the second part of the assignment. `ASTMethodCallVisitor`'s constructor now takes in parameter the list of function names to search for, and will store the results of the search in an a given txt file.
<br><br>
However, we did have to keep an instruction that is specific to sql queries. In section 1.4, we listed the main functions used in php to make db calls. Among them, the following:
<br>

`$object - > mysql - > exec ( * );`

<br><br>
The `exec()` function also exists as a built-in php function (as used in the `./wp-includes/class-snoopy.php` file on line 1018). To differenciate the built-in php function from the sql query, we had to modify the previously generic code and add a check.
<br><br>
The check in question is made on nodes of which the image is 'exec', and makes sure that the parent node is the `->` binary operator, and that it is called on the mysql attribute. The specific steps of the verification are detailed in comments in the code, for better clarity.
</div>

In [ ]:
from typing import List

class ASTMethodCallVisitor:
    def __init__(self, functions_to_search_arr: List[str], output_file):
        self.ast = None
        self.output_file = output_file
        
        # Used to determine if the ast is being visited for the first time, in order to print the name of the file only once in the output file
        self.isFirstVisitToFile = True

        # Used to store all functions to search for while visitng ast. Makes it easier to reuse this class for different use cases.
        self.node_image_arr = functions_to_search_arr

        # Used to print an empty line after an ast in which a function call has been detected (formatting purpose only)
        self.hasFoundCall = False

    def visit(self, ast: AST):
        self.ast = ast
        self.isFirstVisitToFile = True

        # State which file is being visited, whether a call to one of the specified functions (in functions_to_search_arr) has been found or not (only used to kkeep track of progress, to make sure all ast.json files in the directory have been visited)
        print(f"----- Visiting AST from file {self.ast.get_filename()} -----\n")

        # Start by visiting the root node of the ast
        # The private 'visit' function will then recursively visit the children of each node
        self.__visit(self.ast.get_root())

        # After each node of the ast has been visited and if a vulnerable function has been found, print an EOL
        if self.hasFoundCall:
            self.output_file.write("\n")

            # Set the flag back to false for the next file
            self.hasFoundCall = False
        
    def __visit(self, node_id: int):
        # Define the conditions that must be respected for a node to be considered a call to a specified function
        ## The conditions are as follow:
        ### Must be a function/method call (stored in the node's type)
        ### It's name (stored in the node's image) corresponds to one of the names provided in the functions_to_search_arr
        
        # The node's image refers to the name of the function/variable (ex: mysql_query, exec, etc)
        nodeImage = self.ast.get_image(node_id)
        isMethodOrFunctionCall = (self.ast.get_type(node_id) == "FunctionCall" or self.ast.get_type(node_id) == "MethodCall")

        if isMethodOrFunctionCall and nodeImage in self.node_image_arr:
            # If exec function, check if it is called on mysql attribute
            if not (nodeImage == "exec" and not self.__is_sql_exec(node_id)):
                # Set the foundCall flag to true in order to add an empty line in the output file once the ast has been fully visited. Purely for formatting purposes.
                self.hasFoundCall = True

                #  State which file is being visited - only printed once in the output file if a call to a specified function has been detected.
                if self.isFirstVisitToFile:
                    self.output_file.write(f"----- Visiting AST from file {self.ast.get_filename()} -----\n")
                    self.isFirstVisitToFile = False

                # Print the line in the code where a call to the specified method has been made
                self.output_file.write(f"Function '{nodeImage}' is called "
                    f"at line {self.ast.get_position(node_id)[0]}\n")

        # Visit the node's children    
        for child_id in self.ast.get_children(node_id):
            self.__visit(child_id)

    # In this function, we want to make sure that the following pattern is respected:
    '''
    $object -> mysql -> exec ( * );
    AST representation:
                              BinOP (->)
                            /           \        
                      BinOP (->)         exec()
                    /           \    
                this              mysql
    '''
    def __is_sql_exec(self, exec_node_id):
        # Create a lambda function that checks if a given node is indeed the `->` binary operator
        ## We decided to make a lambda function since it is only used in the scope of this function and nowhere else
        is_binOp = lambda node_id: self.ast.get_type(node_id) == "BinOP" and self.ast.get_image(node_id) == "->"

        # Get node's first parent
        parent_id = self.ast.get_parents(exec_node_id)[0]
        parent_binOp_children = []

        # If parent is indeed the `->` binary operator, retrieve all of its children
        ## At this point we've verified this part of the expression: -> exec( * )
        if is_binOp(parent_id):
            # Now to verify the following part of the expression: $object -> mysql, the object being the `this` object.
            ## We need to retrieve the other children of the parent BinOP node
            parent_binOp_children = self.ast.get_children(parent_id)

            # Find the binOp child that is called on the same line as the exec function
            is_exec_same_position = lambda node_id: self.ast.get_position(node_id)[0] == self.ast.get_position(exec_node_id)[0]
            
            # Returns an array of elements satisfying the specified conditions; take the first element (after observing the graphs generated from the .ast.dot files, we deemed it safe to take the first element systematically, 
            # since usually the BinOP nodes only have 2 children; the function, and the variable`object` on which the function is called)
            # We are therefore retrieving the first and only BinOP child.
            ## Adapted from https://stackoverflow.com/questions/7270321/finding-the-index-of-elements-based-on-a-condition-using-python-list-comprehensi
            binOp_child_id = [child_id for child_id in parent_binOp_children if is_binOp(child_id) and is_exec_same_position(child_id)][0]

            # We now retrieve the leftmost children of the subtree we are interested in: $object (which is typically the `this` object in the provided php code) and mysql
            leftmost_children = self.ast.get_children(binOp_child_id)

            # After further observing the graphs generated from the .ast.dot files, we concluded that to do so, we have to check if the node ONLY has the two following children:
            ## type=Variable, image=this
            ## type=Id, image=mysql
            ## We also need to make sure they are all situated on the same line as the exec function
            is_researched_node = lambda node_id, node_type, node_image: self.ast.get_type(node_id) == node_type and self.ast.get_image(node_id) == node_image
            has_this_variable_node = [child_id for child_id in leftmost_children if is_researched_node(child_id, "Variable", "this") and is_exec_same_position(child_id)][0]
            has_mysql_id_node = [child_id for child_id in leftmost_children if is_researched_node(child_id, "Id", "mysql") and is_exec_same_position(child_id)][0]
            
            return len(leftmost_children) == 2 and has_this_variable_node and has_mysql_id_node

## 1.3 For each ast file, load ast in memory and visit nodes in search of function calls

In [ ]:
def findMethodCallInFolder(directory: str, methods_to_find: List[str], output_file):
    # Retrieve filenames of all ast in the specified directory
    astFilenames = get_ast_json_files(directory)

    # Iterate over the filenames array
    for filename in astFilenames:
        # Load ast in memory
        ast = reader.read_ast(filename)

        # Visit currently loaded ast
        visitor = ASTMethodCallVisitor(methods_to_find, output_file)
        visitor.visit(ast)

## 1.4 Specify directory to analyze and functions to find
_**Note:** In php, calls to an SQL db can be done in the following ways:_
```
    mysql_query ( * ) ;
    mysqli_query ( * ) ;
    $object - > execute () ;
    $object - > mysql - > exec ( * ) ;
```  

In [ ]:
# List of all possible db calls in php (based on the above note)
methods_to_find = ["mysql_query", "mysqli_query", "execute", "exec"]
first_directory_to_analyze = "../code_to_analyze/wordpress_ast/"
sql_output_file = create_output_file("calls_to_db_output.txt")

findMethodCallInFolder(first_directory_to_analyze, methods_to_find, sql_output_file)
close_output_file(sql_output_file)

# 2. CVE Detection

<div style="text-align: justify">
For this second part of the lab, we were tasked with finding vulnerable functions pertaining to specific CVEs.
</div>

## 2.1 CVEs to detect (gr.04): 
- [CVE-2017-7189](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2017-7189):
    - **Description**: main/streams/xp_socket.c in PHP 7.x before 2017-03-07 misparses **<u>fsockopen</u>** calls, such as by interpreting fsockopen('127.0.0.1:80', 443) as if the address/port were 127.0.0.1:80:443, which is later truncated to 127.0.0.1:80. This behavior has a security risk if the explicitly provided port number (i.e., 443 in this example) is hardcoded into an application as a security policy, but the hostname argument (i.e., 127.0.0.1:80 in this example) is obtained from untrusted input. 
- [CVE-2019-11039](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2019-11039):
    - **Description**: Function **<u>iconv_mime_decode_headers()</u>** in PHP versions 7.1.x below 7.1.30, 7.2.x below 7.2.19 and 7.3.x below 7.3.6 may perform out-of-buffer read due to integer overflow when parsing MIME headers. This may lead to information disclosure or crash. 
- [CVE-2021-21707](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2021-21707):
    - **Description**: In PHP versions 7.3.x below 7.3.33, 7.4.x below 7.4.26 and 8.0.x below 8.0.13, certain XML parsing functions, like **<u>simplexml_load_file()</u>**, URL-decode the filename passed to them. If that filename contains URL-encoded NUL character, this may cause the function to interpret this as the end of the filename, thus interpreting the filename differently from what the user intended, which may lead it to reading a different file than intended. 

## 2.2 Create new file to store output

In [ ]:
cve_output_file = create_output_file("cve_detection_output.txt")

## 2.3 Find all vulnerable functions in specified folder: 

Based on the descriptions provided in 2.1, each CVE pertains to a specific function. Given that the `findMethodCallInFolder` function implemented in the previous section is generic enough, it can be reused in this section as well, with the appropriate functions specified as parameter.

In [ ]:
directories_to_analyze = ["../code_to_analyze/test_cve/", "../code_to_analyze/wordpress_ast/"]

cve_to_detect = {
    "CVE-2017-7189": ["fsockopen"],
    "CVE-2019-11039": ["iconv_mime_decode_headers"],
    "CVE-2021-21707": ["simplexml_load_file"]
}

for directory in directories_to_analyze:
    cve_output_file.write(f"--------------- Directory: {directory} ---------------\n")
    for cve in cve_to_detect:
        cve_output_file.write(f"--------------- {cve} ---------------\n")
        findMethodCallInFolder(directory, cve_to_detect[cve], cve_output_file)
        cve_output_file.write(f"\n")

close_output_file(cve_output_file)

# Difficulties

<div style="text-align: justify">

As previously explained, the first difficulty we encountered was the `exec()` method being a built-in php method, unlike the other methods we had to find within the code. We had to observe the generated .dot graphs in order to gain a better understanding of the asts and find out how to make sure we were detecting the correct `exec()`, which needs to be called on the `mysql` attribute.

<br>

The second difficulty or rather, weakness, in our current code, is the code presented in section 2.3. As it is, the code traverses each directory 3 times; once for each cve. It takes time (approx. 2:20 min on average). We decided to do it this way in order to properly separate and identified what CVE was detected and where in the output file. The alternative would have been to feed all vulnerable functions to the `findMethodCallInFolder()` method in a single array, regardless of the associated CVE, and simply print the function and where it was found, without the CVE header.

</div>